# LeNet CNN

In [58]:
import tensorflow as tf
import numpy as np




<img src="optimizer_graph.png" width="200px">

## Reading Data

We are reading MNIST data from http://yann.lecun.com/exdb/mnist/


In [60]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
model_path = "/Users/omerorhan/Desktop/UCSC/Deep Learning" \
             "/DeepLearningUCSC/venv/homework/Assignments/model.ckpt"


Extracting /tmp/data/train-images-idx3-ubyte.gz


Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [61]:
print(mnist.train.num_examples) # Number of training data
print(mnist.test.num_examples) # Number of test data


55000
10000


## Setting hyper-parameters

In [62]:
# architecture hyper-parameter
noofdatapoints = mnist.train.num_examples

learningrate = 0.001
nepochs = 10
batch_size = 512
noofbatches = 10#noofdatapoints//batch_size
print("no of batches =", noofbatches)

n_input = 784 # 28x28 image
n_classes = 10 # 1 for each digit [0-9]
dropout = 0.75 

no of batches = 10


## Defining x and y as placeholders

In [63]:
X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)
print(X.shape, Y.shape)

(?, 784) (?, 10)


## Defining model using conv2d, Relu and Maxpooling

In [64]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [65]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [66]:
def conv_net(x, weights, biases, dropout):
    # reshape input to 28x28 size
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution layer 1
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max pooling
    conv1 = maxpool2d(conv1, k=2)

    # Convolution layer 2
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max pooling
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

## Initializing Weights and Biases

In [67]:


weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

## Define cost, optimizer and accuracy

In [80]:
# Create the model
model = conv_net(X, weights, biases, keep_prob)
print(model)
# Define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
train_min = tf.train.AdamOptimizer(learning_rate=learningrate).minimize(loss)

# Evaluate model
correct_model = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_model, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

weights_saver = tf.train.Saver(var_list=weights)


Tensor("Add_25:0", shape=(?, 10), dtype=float32)


## Running the session 
This will run the graph and use all the tensors that were previously defined

In [81]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(nepochs):
        for _ in range(noofbatches):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            #batch_y = batch_y.astype(np.float32)
            # Use training data for optimization
            sess.run(train_min, feed_dict={X:batch_x, Y:batch_y, keep_prob: dropout})
        # Validate after every epoch
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        losscalc, accuracycalc = sess.run([loss, accuracy], 
                                          feed_dict={X:batch_x, Y:batch_y, keep_prob: 1.0})
        print("Epoch: %d, Loss: %0.4f, Accuracy: %0.4f"%(epoch, losscalc, accuracycalc))
            
    # When the training is complete and you are happy with the result

    accuracycalc = sess.run(accuracy, 
                            feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1.0})
    print("Testing accuracy: %0.4f"%(accuracycalc))
    
    weights_saver.save(sess,model_path)


Epoch: 0, Loss: 27867.2969, Accuracy: 0.2676


Epoch: 1, Loss: 10045.0215, Accuracy: 0.5508


Epoch: 2, Loss: 6153.5054, Accuracy: 0.6895


Epoch: 3, Loss: 4225.5430, Accuracy: 0.7910


Epoch: 4, Loss: 3483.7827, Accuracy: 0.8066


...

Epoch: 6, Loss: 2615.6829, Accuracy: 0.8770


Epoch: 7, Loss: 2215.6050, Accuracy: 0.8730


Epoch: 8, Loss: 1688.7600, Accuracy: 0.8906


Epoch: 9, Loss: 1741.3367, Accuracy: 0.8867


Testing accuracy: 0.8991


# LeNet CNN with Fashion MNIST


In [71]:
from tensorflow.examples.tutorials.mnist import input_data
fmnist = input_data.read_data_sets('data/fashion', one_hot=True)


Extracting data/fashion/train-images-idx3-ubyte.gz


Extracting data/fashion/train-labels-idx1-ubyte.gz
Extracting data/fashion/t10k-images-idx3-ubyte.gz
Extracting data/fashion/t10k-labels-idx1-ubyte.gz


In [72]:
# architecture hyper-parameter
noofdatapoints = fmnist.train.num_examples

learningrate = 0.001
nepochs = 10
batch_size = 512
noofbatches = 10#noofdatapoints//batch_size
print("no of batches =", noofbatches)

n_input = 784 # 28x28 image
n_classes = 10 # 1 for each digit [0-9]
dropout = 0.75 

X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)
print(X.shape, Y.shape)

def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def conv_net(x, weights, biases, dropout):
    # reshape input to 28x28 size
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution layer 1
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max pooling
    conv1 = maxpool2d(conv1, k=2)

    # Convolution layer 2
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max pooling
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out


no of batches = 10
(?, 784) (?, 10)


In [82]:



weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}
biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

INFO:tensorflow:Restoring parameters from /Users/omerorhan/Desktop/UCSC/Deep Learning/DeepLearningUCSC/venv/homework/Assignments/model.ckpt


Model restored.


In [74]:
# Create the model
model = conv_net(X, weights, biases, keep_prob)
print(model)
# Define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
train_min = tf.train.AdamOptimizer(learning_rate=learningrate).minimize(loss)

# Evaluate model
correct_model = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_model, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

Tensor("Add_19:0", shape=(?, 10), dtype=float32)


In [86]:
with tf.Session() as sess:
    sess.run(init)

    weights_saver.restore(sess, "/Users/omerorhan/Desktop/UCSC/Deep Learning/DeepLearningUCSC/venv"
                        "/homework/Assignments/model.ckpt")
    print("Model restored.")
        
    for epoch in range(nepochs):
        for _ in range(noofbatches):
            batch_x, batch_y = fmnist.train.next_batch(batch_size)
            #batch_y = batch_y.astype(np.float32)
            # Use training data for optimization
            sess.run(train_min, feed_dict={X:batch_x, Y:batch_y, keep_prob: dropout})
        # Validate after every epoch

        batch_x, batch_y = fmnist.train.next_batch(batch_size)
        losscalc, accuracycalc = sess.run([loss, accuracy], 
                                          feed_dict={X:batch_x, Y:batch_y, keep_prob: 1.0})
        print("Epoch: %d, Loss: %0.4f, Accuracy: %0.4f"%(epoch, losscalc, accuracycalc))
            
    # When the training is complete and you are happy with the result
    accuracycalc = sess.run(accuracy, 
                            feed_dict={X: fmnist.test.images, Y: fmnist.test.labels, keep_prob: 1.0})
    print("Testing accuracy: %0.4f"%(accuracycalc))
    #print(sess.run(weightss))


INFO:tensorflow:Restoring parameters from /Users/omerorhan/Desktop/UCSC/Deep Learning/DeepLearningUCSC/venv/homework/Assignments/model.ckpt


Model restored.


Epoch: 0, Loss: 2511.9146, Accuracy: 0.8691


Epoch: 1, Loss: 2894.0081, Accuracy: 0.8574


Epoch: 2, Loss: 1720.3448, Accuracy: 0.9082


...

Epoch: 6, Loss: 1347.9968, Accuracy: 0.9219


Epoch: 7, Loss: 1177.0840, Accuracy: 0.9141


Epoch: 8, Loss: 1271.6121, Accuracy: 0.9316


Epoch: 9, Loss: 1061.7227, Accuracy: 0.9199


Testing accuracy: 0.9382
